In [56]:
#pragma cling load("./build/libkastore.so")
#pragma cling load("./build/libtskit.so")
#pragma cling add_include_path("./tskit/c")
#pragma cling add_include_path("./kastore/c")

#include "./kastore/c/kastore.h"
#include "./tskit/c/tskit.h"

#include <tskit.h>
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <vector>

#define check_tsk_error(val)                                                            \
    if (val < 0) {                                                                      \
        fprintf(stderr, "line %d: %s", __LINE__, tsk_strerror(val));                    \
        exit(EXIT_FAILURE);                                                             \
    }

# Initialising mock data

In [4]:
// node collection

In [6]:
void sleepy_create_mock_nodes(tsk_node_table_t *self, int length=10) {
    for (int i=0; i<length; i++) {
        tsk_node_table_add_row(self, TSK_NODE_IS_SAMPLE, i, TSK_NULL, TSK_NULL, NULL, 0);
    }
}

In [7]:
// for testing
tsk_node_table_t nodes;
tsk_node_table_init(&nodes, 0);
sleepy_create_mock_nodes(&nodes)
tsk_node_table_print_state(&nodes, stdout)

# Initialising table data

In [22]:
void sleepy_init_tables(tsk_table_collection_t *self, int N=3, int m=1, int gen=1) {
    for (int i=0; i<m; i++) {
        for (int j=0; j<2*N; j++) {
            tsk_node_table_add_row(&self->nodes, TSK_NODE_IS_SAMPLE, (double) gen,
                                   TSK_NULL, TSK_NULL, NULL, 0);
        }
        gen++;
    }
}

In [23]:
tsk_table_collection_t tables;
tsk_table_collection_init(&tables, 0);

sleepy_init_tables(&tables, 3, 1, gen);
tables.nodes.num_rows

6

In [14]:
tsk_node_table_print_state(&tables.nodes, stdout)

# Reversing time function

In [42]:
void s3_reverse_time(tsk_node_table_t &nodes, int begin, int end = 0)
{
    std::vector<double> time_before; time_before.reserve(begin);
    std::vector<double> time_mid; time_before.reserve(begin+nodes.num_rows-end);
    std::vector<double> time_after;
    
    if (end == 0) end = nodes.num_rows;
    
    for (int i=0; i<begin; i++) time_before.emplace_back(nodes.time[i]);
    
    double max_element{0};
    for (int i=begin; i<end; i++) {
        if (nodes.time[i] >= max_element) {
            max_element = nodes.time[i];
        }
    }
    
    for (int i=begin; i<end; i++) {
        time_mid.emplace_back((nodes.time[i] - max_element) * (-1));
    }
    
    for (int i=end; i<nodes.num_rows; i++) {
        time_after.emplace_back(nodes.time[i]);
    }
    
    time_before.insert(std::end(time_before), std::begin(time_mid), std::end(time_mid));
    time_before.insert(std::end(time_before), std::begin(time_after), std::end(time_after));
    
    for (int i=0; i<nodes.num_rows; i++) {
        nodes.time[i] = time_before[i];
    }
    
}

In [ ]:
// test completed